In [48]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as st
import math as mth
import seaborn as sns
from plotly import graph_objects as go
import plotly.express as px
import sys
import getopt
import re
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError

In [49]:
'''
Analisar argumentos da linha de comando para 
retornar o caminho do arquivo.

Retorna:
file_path (str): o caminho para o arquivo 
fornecido pelo usuário.
'''

def parse_arguments():
    unixOptions = 'f:' # argumento no formato unix e ira retornar '-f'
    gnuOptions = ['file='] # argumento no formato gnu e ira retornar '--file='

    fullCmdArguments = sys.argv
    argumentList = fullCmdArguments[1:] # excluir o nome do script

    try:
        arguments, valeus = getopt.getopt(argumentList, unixOptions, gnuOptions)
    except getopt.error as err:
        print(str(err))
        sys.exit(2)
    
    file_path = ''

    for currentArgument, currentValue in arguments:
        if currentArgument in ('-f', '--file'):
            file_path = currentValue
            
    return file_path


In [50]:
unixOptions = 'f:'
gnuOptions = ['file=']

fullCmdArguments = sys.argv
argumentList = fullCmdArguments[1:] # excluir o nome do script

print(fullCmdArguments)
print()
print(argumentList)

['c:\\Users\\carlo\\anaconda3\\Lib\\site-packages\\ipykernel_launcher.py', '--f=c:\\Users\\carlo\\AppData\\Roaming\\jupyter\\runtime\\kernel-v3ee2594d20f59401162676b323c90e71c2979f7df.json']

['--f=c:\\Users\\carlo\\AppData\\Roaming\\jupyter\\runtime\\kernel-v3ee2594d20f59401162676b323c90e71c2979f7df.json']


In [51]:
try:
    arguments, valeus = getopt.getopt(argumentList, unixOptions, gnuOptions)
    print(arguments, valeus)
except getopt.error as err:
    print(str(err))
    sys.exit(2)



[('--file', 'c:\\Users\\carlo\\AppData\\Roaming\\jupyter\\runtime\\kernel-v3ee2594d20f59401162676b323c90e71c2979f7df.json')] []


In [52]:
file_path = ''

for currentArgument, currentValue in arguments:
    if currentArgument in ('-f', '--file'):
        file_path = currentValue
        
print(file_path)



c:\Users\carlo\AppData\Roaming\jupyter\runtime\kernel-v3ee2594d20f59401162676b323c90e71c2979f7df.json


In [53]:
'''
Extrair o ano do nome do caminho.

Retorna:
o ano do documento com os dados.
'''
# Dividir o caminho do arquivo em partes

def extract_year_from_path(file_path):
    year = file_path.split('/')[-1].split('.')[0][-4:]
    return year

In [54]:
# y1 = file_path.split('/')[-1]
# y2 = file_path.split('/')[-1].split('.')[0]
# y3 = file_path.split('/')[-1][-4:]

# print(year)
# print(y1)
# print(y2)
# print(y3)

In [55]:
'''
Valida se os dados que queremos armazenar já existem no banco de dados. Para fazer isso:
1. Use o objeto "engine" para se conectar ao banco de dados.
2. Crie uma consulta SQL para verificar se há algum registro em "table_name" correspondente ao "year" (ano) fornecido.
3. Execute a consulta SQL.
        
Retorna:
True se os dados já estão armazenados no seu banco de dados. Caso contrário, retorna False. Analisar
outros resultados possíveis ao consultar a tabela.
'''

def data_already_exist(engine, table_name, year):
    try:
        with engine.connect() as connection:
            query = text(f'SELECT * FROM {table_name} WHERE ANO_EGRESO={year}')
            result = connection.execute(query)
            exists = result.fetchone() is not None
    except OperationalError as e:
        exists = False
        
    return exists

In [56]:
# Carregar os dados em um DataFrame para operações futuras

def load_data(file_path):
    df = pd.read_csv(file_path, encoding='latin1', sep=';')
    return df

In [57]:
'''
Args:
df (pd.DataFrame): o DataFrame de entrada.
threshold (float): a proporção de colunas que podem conter '*' antes que 
a linha seja removida.
        
Pré-processar o DataFrame removendo as linhas em que a maioria das colunas contêm o 
caractere '*'. Você também precisa padronizar os tipos de dados de colunas de acordo com a
estrutura de dados apropriada. Inspecionar arquivos CSV diferentes para analisar possíveis problemas. Para fazer isso:
            
1. Calcule o número de colunas.
2. Determine quantos caracteres '*' são permitidos por linha com base no limiar.
3. Filtre as linhas que ultrapassaram o número permitido de '*'.
4. Converta colunas específicas para o tipo inteiro.
5. Renomeie as colunas usando uma lista predefinida de nomes novos.
6. Retorne o DataFrame limpo e formatado.
                
Retorna:
pd.DataFrame: o DataFrame limpo.
'''

def preprocess_data(df, threshold=0.5): # versão antiga: def preprocess_data(df, engine, threshold=0.5):
    # Calcular o número de colunas
    num_columns = len(df.columns) # ou df.shape[1]

     # Determinar o número de caracteres '*' permitidos com base no limiar
    allowed_stars = int(num_columns * threshold)

    # Filtrar as linhas em que o número de caracteres '*' excede o limiar permitido
    cleaned_df = df[df.apply(lambda x: (x == '*').sum() <= allowed_stars, axis=1)]

    # Formato de dados
    cleaned_df.loc[:,'COMUNA_RESIDENCIA'] = cleaned_df['COMUNA_RESIDENCIA'].astype(int)
    cleaned_df.loc[:,'REGION_RESIDENCIA'] = cleaned_df['REGION_RESIDENCIA'].astype(int)
    cleaned_df.loc[:,'ANO_EGRESO'] = cleaned_df['ANO_EGRESO'].astype(int)

    # Renomear as colunas
    new_column_names = ['PERTENENCIA_ESTABLECIMIENTO_SALUD', 'SEXO', 'GRUPO_EDAD', 'ETNIA',
       'GLOSA_PAIS_ORIGEN', 'COMUNA_RESIDENCIA', 'GLOSA_COMUNA_RESIDENCIA',
       'REGION_RESIDENCIA', 'GLOSA_REGION_RESIDENCIA', 'PREVISION',
       'GLOSA_PREVISION', 'ANO_EGRESO', 'DIAG1', 'DIAG2', 'DIAS_ESTADA',
       'CONDICION_EGRESO', 'INTERV_Q', 'PROCED']
    old_column_names = cleaned_df.columns    
    
    column_mapping = dict(zip(old_column_names, new_column_names))
    cleaned_df.rename(columns=column_mapping, inplace=True)

    return cleaned_df
    

In [58]:
'''
Criar uma conexão ao banco de dados com "sqlite:///"
1. Crie uma string de conexão usando um nome de banco de dados conveniente.  Por exemplo,
"sqlite:///{nome do seu banco de dados}.db". Armazene-a em uma variável.
2. Inicie o mecanismo do SQLAlchemy para o banco de dados chamando create_engine(). Passe
a variável anterior como um parâmetro dessa função.
3. Imprima uma mensagem de confirmação de conexão.
4. Retorne o mecanismo do SQLAlchemy para mais interações com o banco de dados.
'''

def create_db_engine(db_name): # código "sqlite:///C:\Users\carlo\Downloads\GIT\Tripleten\sprint_12_saude_chile.db"
    connection_string = f'sqlite:///{db_name}'
    engine = create_engine(connection_string)
    print(f'[INFO]: Conexão verificada: {connection_string}')
    return engine




In [59]:
'''
Salvar o DataFrame (df) limpo no seu banco de dados.
1. O DataFrame é inserido em uma tabela SQL especificada por table_name.
2. Se a tabela já existir, novos dados serão anexados.
3. O índice do DataFrame é excluído das colunas da tabela.
4. A conexão ao banco de dados é gerenciada usando o mecanismo fornecido.
'''

def save_to_database(df, engine, table_name):
    df.to_sql(name=table_name, con=engine, if_exists='append', index=False)
    #return



In [61]:
# código para testar seu pipeline

def validate_data(engine, table_name):
    with engine.connect() as connection:
        query = text(f'SELECT ANO_EGRESO, count(*) FROM {table_name} GROUP BY ANO_EGRESO')
        result = connection.execute(query)

        rows = result.fetchall()
        for row in rows[:100]:
            print(row)

# Analisar o caminho do arquivo usando os argumentos da linha de comando
if __name__=="__main__":
    file_path = parse_arguments()
    table_name = 'egresos_pacientes'

    # Carregar o banco de dados no seu sistema
    engine = create_db_engine('database/ministerio_de_salud_chile.db')
    print('[INFO]: Conexão com o banco de dados')

    if file_path:
        print(f"Caminho do arquivo: {file_path}")
        year = extract_year_from_path(file_path)

        # Carregar seu arquivo CSV em um DataFrame do Pandas
        raw_data = load_data(file_path)
        print('[INFO]: Carregar os dados como um DataFrame do Pandas')
        print(raw_data.columns.tolist())
        
        # Verificar se os dados já estão no banco de dados
        if data_already_exist(engine, table_name, year):
            print(f"Os dados já existem no banco de dados. Nenhuma ação deve ser tomada.")   
        else:
            raw_data = preprocess_data(raw_data)
            print('[INFO]: Pré-processar os dados')
            
            # Salvar os dados em uma nova tabela dentro do banco de dados existente    
            save_to_database(raw_data, engine, table_name)
            print('[INFO]: Carregar os dados no banco de dados')
    else:
        print('Nenhum caminho foi fornecido')
    
    validate_data(engine, table_name)
    # return




[INFO]: Conexão verificada: sqlite:///database/ministerio_de_salud_chile.db
[INFO]: Conexão com o banco de dados
Caminho do arquivo: c:\Users\carlo\AppData\Roaming\jupyter\runtime\kernel-v3ee2594d20f59401162676b323c90e71c2979f7df.json
[INFO]: Carregar os dados como um DataFrame do Pandas
['{"key":"dd7f159f-fbdd-4139-9fd3-3b2d6eceab55","signature_scheme":"hmac-sha256","transport":"tcp","ip":"127.0.0.1","hb_port":9000,"control_port":9001,"shell_port":9002,"stdin_port":9003,"iopub_port":9004,"kernel_name":"python3124jvsc74a57bd08e76f8642360a3358ab06c318ab18c1161224eb3836609d5c16edea6f6e43dba"}']


KeyError: 'COMUNA_RESIDENCIA'

In [ ]:
'''
Você pode usar a seguinte linha de comando para executar seu código e armazenar um arquivo. Por exemplo:

python test.py -f dbs/EGRESOS_2020/EGRE_DATOS_ABIERTOS_2020.csv
'''

# "C:\Users\carlo\Downloads\GIT\Tripleten\sprint_12_saude_chile\EGRE_DATOS_ABIERTOS_2018.csv"

In [ ]:
# parse_arguments()
# extract_year_from_path(file_path)
# data_already_exists(engine, table_name, year)
# load_data(file_path)
# preprocess_data(df, threshold=0.5)
# create_db_engine(db_name)
# save_to_database(df, engine, table_name)
# validate_data(engine, table_name)